In [1]:
from src.chromosome import Chromosome
import numpy as np

In [2]:
def fitnessFunction(x, y):
    result = (
        np.sin(x) * np.exp((1 - np.cos(y))**2) +
        np.cos(y) * np.exp((1 - np.sin(x))**2) +
        (x - y)**2
    )
    return round(result, 2)

In [3]:
def linearMappingFunction(i, N):
    min = 0
    max = 100
    result = min + (max - min) * ((N - i) / (N - 1))
    return round(result, 2)

In [4]:
def generateChromosome():
    return Chromosome(
        round(np.random.uniform(-10, 10), 2),
        round(np.random.uniform(-10, 10), 2)
    )

In [5]:
def sortChromosomes(chromosomes):
    return sorted(chromosomes, key=lambda chromosome: chromosome.fitness)

In [6]:
N = 100

chromosome = []
for i in range(N):
    chromosome.append(generateChromosome())
    chromosome[i].calculateFitness(fitnessFunction)

chromosome = sortChromosomes(chromosome)

for i in range(N):
    chromosome[i].calculateMapFitness(linearMappingFunction, i + 1, N)

for i in chromosome:
    print(i.__str__())

x: 5.31     y: 9.73     Fitness: -44.89     Mapped Fitness: 100.0
x: 4.45     y: -3.1     Fitness: -43.17     Mapped Fitness: 98.99
x: -7.68     y: -2.43     Fitness: -32.98     Mapped Fitness: 97.98
x: 4.87     y: 4.35     Fitness: -24.34     Mapped Fitness: 96.97
x: -1.51     y: -1.8     Fitness: -16.73     Mapped Fitness: 95.96
x: 9.61     y: 9.71     Fitness: -12.46     Mapped Fitness: 94.95
x: 9.77     y: 8.59     Fitness: -8.16     Mapped Fitness: 93.94
x: 0.79     y: 1.02     Fitness: 1.51     Mapped Fitness: 92.93
x: 0.12     y: 0.57     Fitness: 2.15     Mapped Fitness: 91.92
x: 6.04     y: 5.08     Fitness: 2.23     Mapped Fitness: 90.91
x: -0.03     y: -1.83     Fitness: 2.35     Mapped Fitness: 89.9
x: 6.59     y: 5.67     Fitness: 2.49     Mapped Fitness: 88.89
x: -0.12     y: -0.58     Fitness: 3.02     Mapped Fitness: 87.88
x: 0.12     y: -1.56     Fitness: 3.16     Mapped Fitness: 86.87
x: -5.85     y: -4.22     Fitness: 5.67     Mapped Fitness: 85.86
x: -9.22     y: -7